# Synthetic Image Generation Tests

## Overview
This notebook generates synthetic test images and encodes them using standard Python libraries (imagecodecs). The generated images can be used as test inputs for jpegexp-rs verification and comparison.

## Purpose
- Generate reproducible synthetic test images
- Create baseline encoded files using reference implementations
- Provide test images for codec verification workflows

## Test Images Generated
1. **Grayscale Gradient**: 256x256 horizontal gradient (black to white)
2. **RGB Pattern**: 256x256 vertical stripes (Red, Green, Blue)

## Codecs Used
- **JPEG** (Baseline): Lossy compression via imagecodecs
- **JPEG-LS**: Lossless compression via imagecodecs/CharLS
- **JPEG 2000**: Lossy compression via imagecodecs/OpenJPEG

## Output
All encoded images are saved to `tests/out/imagetests/` directory.


In [ ]:
# ============================================================================
# Synthetic Image Generation and Encoding Tests
# ============================================================================
# This notebook generates synthetic test images and encodes them with various
# codecs using standard Python libraries (imagecodecs). It serves as a baseline
# for creating test images that can be used with jpegexp-rs.
# ============================================================================

%matplotlib inline

# Standard library imports
import os
import subprocess
from io import BytesIO

# Third-party imports
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import imagecodecs  # Provides JPEG, JPEG-LS, and JPEG 2000 encoders

# Create output directory for generated test files
output_dir = os.path.join("..", "tests", "out", "imagetests")
os.makedirs(output_dir, exist_ok=True)


# NOTE: This notebook uses imagecodecs for encoding, not jpegexp-rs CLI
# The binary path below is kept for potential future integration
# but is not currently used in this notebook

possible_paths = [
    os.path.join("..", "target", "release", "jpegexp.exe"),
    os.path.join("target", "release", "jpegexp.exe"),
    os.path.join("..", "target", "debug", "jpegexp.exe"),
    os.path.join("target", "debug", "jpegexp.exe")
]
BINARY_PATH = None
for p in possible_paths:
    if os.path.exists(p):
        BINARY_PATH = os.path.abspath(p)
        break

if not BINARY_PATH:
    print(f"NOTE: jpegexp-rs binary not found (not required for this notebook)")
    print(f"  Searched in: {possible_paths}")
else:
    print(f"jpegexp-rs binary found: {BINARY_PATH} (not used in this notebook)")


In [ ]:
# Generate a 256x256 synthetic grayscale gradient image
# Horizontal gradient: left side is black (0), right side is white (255)
gradient = np.tile(np.linspace(0, 255, 256, dtype=np.uint8), (256, 1))

# Convert to PIL Image for visualization
img = Image.fromarray(gradient, mode='L')

# Display the generated image
plt.figure(figsize=(8, 6))
plt.imshow(img, cmap='gray')
plt.title("256x256 Grayscale Gradient (Test Image)")
plt.axis('off')
plt.tight_layout()
plt.show()

# Encode the image with multiple codecs using imagecodecs
print("Encoding with multiple codecs...")
jpeg_ls_bytes = imagecodecs.jpegls_encode(np.array(img))  # JPEG-LS (lossless)
jpeg_bytes = imagecodecs.jpeg_encode(np.array(img))      # JPEG 1 (lossy)
jpeg_2k_bytes = imagecodecs.jpeg2k_encode(np.array(img), level=5)  # JPEG 2000 (lossy, quality level 5)

# Save encoded files
jpeg_ls_path = os.path.join(output_dir, "gradient.jls")
jpeg_path = os.path.join(output_dir, "gradient.jpg")
jpeg_2k_path = os.path.join(output_dir, "gradient.jp2")

with open(jpeg_ls_path, "wb") as f:
    f.write(jpeg_ls_bytes)
with open(jpeg_path, "wb") as f:
    f.write(jpeg_bytes)
with open(jpeg_2k_path, "wb") as f:
    f.write(jpeg_2k_bytes)

print(f"\nEncoded images saved:")
print(f"  JPEG-LS:    {jpeg_ls_path} ({len(jpeg_ls_bytes)} bytes)")
print(f"  JPEG:       {jpeg_path} ({len(jpeg_bytes)} bytes)")
print(f"  JPEG 2000:  {jpeg_2k_path} ({len(jpeg_2k_bytes)} bytes)")

In [ ]:
# Generate a 256x256 synthetic RGB pattern image
# Three vertical stripes: Red (left), Green (middle), Blue (right)
rgb_pattern = np.zeros((256, 256, 3), dtype=np.uint8)
stripe_width = 256 // 3
rgb_pattern[:, :stripe_width, 0] = 255   # Red stripe (left third)
rgb_pattern[:, stripe_width:2*stripe_width, 1] = 255  # Green stripe (middle third)
rgb_pattern[:, 2*stripe_width:, 2] = 255  # Blue stripe (right third)

# Convert to PIL Image for visualization
img_rgb = Image.fromarray(rgb_pattern, mode='RGB')

# Display the generated image
plt.figure(figsize=(8, 6))
plt.imshow(img_rgb)
plt.title("256x256 RGB Pattern (R/G/B Stripes)")
plt.axis('off')
plt.tight_layout()
plt.show()

# Encode the RGB image with multiple codecs using imagecodecs
print("Encoding RGB image with multiple codecs...")
jpeg_ls_rgb_bytes = imagecodecs.jpegls_encode(np.array(img_rgb))  # JPEG-LS
jpeg_rgb_bytes = imagecodecs.jpeg_encode(np.array(img_rgb))        # JPEG 1
jpeg_2k_rgb_bytes = imagecodecs.jpeg2k_encode(np.array(img_rgb), level=5)  # JPEG 2000

# Save encoded files
jpeg_ls_rgb_path = os.path.join(output_dir, "rgb_pattern.jls")
jpeg_rgb_path = os.path.join(output_dir, "rgb_pattern.jpg")
jpeg_2k_rgb_path = os.path.join(output_dir, "rgb_pattern.jp2")

with open(jpeg_ls_rgb_path, "wb") as f:
    f.write(jpeg_ls_rgb_bytes)
with open(jpeg_rgb_path, "wb") as f:
    f.write(jpeg_rgb_bytes)
with open(jpeg_2k_rgb_path, "wb") as f:
    f.write(jpeg_2k_rgb_bytes)

print(f"\nEncoded RGB images saved:")
print(f"  JPEG-LS:    {jpeg_ls_rgb_path} ({len(jpeg_ls_rgb_bytes)} bytes)")
print(f"  JPEG:       {jpeg_rgb_path} ({len(jpeg_rgb_bytes)} bytes)")
print(f"  JPEG 2000:  {jpeg_2k_rgb_path} ({len(jpeg_2k_rgb_bytes)} bytes)")